In [1]:
using CSV, DataFrames, Statistics

In [2]:
df1 = CSV.read("pattern_counts_allfeat.csv", DataFrame) #Before one-hot-encoding
select!(df1, [:Name, :n, :p, :Num_Patterns, :p_miss])
rename!(df1, :p => :p_all, :p_miss => :p_miss_all)

,Name,n,p_all,Num_Patterns,p_miss_all
,String,Int64,Int64,Int64,Int64
1,COUNT-loomis,384,9,4,9
2,Ecdat-MCAS,180,13,3,2
3,Ecdat-Males,4360,37,2,4
4,Ecdat-Mofa,50,53,2,1
5,Ecdat-RetSchool,3078,37,8,4
6,Ecdat-Schooling,3010,51,9,10
7,MASS-Cars93,93,64,3,2
8,MASS-Pima.tr2,300,7,6,3
9,MASS-survey,237,24,8,23


In [3]:
df2 = CSV.read("pattern_counts_numonly.csv", DataFrame) #After one-hot-encoding
select!(df2, [:Name, :p, :p_miss])
rename!(df2, :p => :p_num, :p_miss => :p_miss_num)

,Name,p_num,p_miss_num
,String,Int64,Int64
1,COUNT-loomis,12,0
2,Ecdat-MCAS,13,2
3,Ecdat-Males,38,0
4,Ecdat-Mofa,53,1
5,Ecdat-RetSchool,37,4
6,Ecdat-Schooling,53,2
7,MASS-Cars93,64,2
8,MASS-Pima.tr2,7,3
9,MASS-survey,29,4


In [4]:
df = innerjoin(df1, df2, on=:Name, makeunique=true)
df[!,:missing_num] .= df[:,:p_miss_num]
df[!,:missing_cat] .= df[:,:p_miss_all] .- df[:,:p_miss_num]
sort!(df, [:missing_num, :missing_cat, :Name])

,Name,n,p_all,Num_Patterns,p_miss_all,p_num
,String,Int64,Int64,Int64,Int64,Int64
1,Ecdat-Males,4360,37,2,4,38
2,mushroom,8124,116,2,4,117
3,post-operative-patient,90,23,2,4,24
4,breast-cancer,286,41,3,7,43
5,heart-disease-cleveland,303,28,3,8,30
6,COUNT-loomis,384,9,4,9,12
7,Zelig-coalition2,314,24,2,14,25
8,shuttle-landing-control,15,16,6,16,21
9,trains,10,77,4,22,87


In [5]:
pb_datasets = ["cylinder-bands", "ozone-level-detection-eight", "ozone-level-detection-one", "thyroid-disease-thyroid-0387", "trains",
                "credit-approval", "Ecdat-Mofa", "sleep"]

8-element Vector{String}:
 "cylinder-bands"
 "ozone-level-detection-eight"
 "ozone-level-detection-one"
 "thyroid-disease-thyroid-0387"
 "trains"
 "credit-approval"
 "Ecdat-Mofa"
 "sleep"

In [6]:
filter!(t -> t[:Name] ∉ pb_datasets, df)

,Name,n,p_all,Num_Patterns,p_miss_all,p_num
,String,Int64,Int64,Int64,Int64,Int64
1,Ecdat-Males,4360,37,2,4,38
2,mushroom,8124,116,2,4,117
3,post-operative-patient,90,23,2,4,24
4,breast-cancer,286,41,3,7,43
5,heart-disease-cleveland,303,28,3,8,30
6,COUNT-loomis,384,9,4,9,12
7,Zelig-coalition2,314,24,2,14,25
8,shuttle-landing-control,15,16,6,16,21
9,congressional-voting-records,435,32,60,32,48


In [7]:
s = ""
for i in 1:nrow(df)
    dname = df[i,:Name]
    if df[i,:missing_num] == 0 
        s *= df[i,:Name]*" & "
        s *= string(df[i,:n]," & ")
        s *= string(df[i,:p_all]," & ")
        s *= string(df[i,:missing_num]," & ")
        s *= string(df[i,:missing_cat]," & ")
        s *= string(df[i,:Num_Patterns]," & ")
        s *= string(df[i,:p_num], " & ")
        try 
            target_list = names(CSV.read("../datasets/"*dname*"/Y.csv", missingstrings=["", "NaN"], DataFrame))

            Y = []
            if length(target_list) <= 2
                target_name = setdiff(target_list, [:Id])[1]
                Y = CSV.read("../datasets/"*dname*"/Y.csv", missingstrings=["", "NaN"], DataFrame)[:,target_name]
            else
                setdiff!(target_list, [:Id, :target])
                sort!(target_list)
                Y = CSV.read("../datasets/"*dname*"/Y.csv", missingstrings=["", "NaN"], DataFrame)[:,target_list[1]]
            end
            if eltype(Y) ∉ [Float64, Int64, Union{Float64,Missing}, Union{Int64,Missing}]
                using StatsBase
                cm = countmap(Y)
                level = collect(keys(cm))[argmax(collect(values(cm)))]
                Y = 1.0 .* (Y .== level)
            end
            ind_availtarget = .!ismissing.(Y)
            Y = 1.0 .* Y[ind_availtarget] #Remove missing entries before converting to Float64 !

            s *= string(length(unique(Y)) == 2 ? "bin." : "cont.")

        catch 
            s *= string("\\texttt{NA}")
        end 

        s *= " \\\\ \n"
    end
end
print(s)

Ecdat-Males & 4360 & 37 & 0 & 4 & 2 & 38 & cont. \\ 
mushroom & 8124 & 116 & 0 & 4 & 2 & 117 & bin. \\ 
post-operative-patient & 90 & 23 & 0 & 4 & 2 & 24 & bin. \\ 
breast-cancer & 286 & 41 & 0 & 7 & 3 & 43 & bin. \\ 
heart-disease-cleveland & 303 & 28 & 0 & 8 & 3 & 30 & bin. \\ 
COUNT-loomis & 384 & 9 & 0 & 9 & 4 & 12 & cont. \\ 
Zelig-coalition2 & 314 & 24 & 0 & 14 & 2 & 25 & \texttt{NA} \\ 
shuttle-landing-control & 15 & 16 & 0 & 16 & 6 & 21 & bin. \\ 
congressional-voting-records & 435 & 32 & 0 & 32 & 60 & 48 & bin. \\ 
lung-cancer & 32 & 157 & 0 & 33 & 3 & 159 & bin. \\ 
soybean-large & 307 & 98 & 0 & 98 & 8 & 132 & bin. \\ 


┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead

In [8]:
s = ""
for i in 1:nrow(df)
    dname = df[i,:Name]
    if df[i,:missing_cat] == 0 
        s *= df[i,:Name]*" & "
        s *= string(df[i,:n]," & ")
        s *= string(df[i,:p_all]," & ")
        s *= string(df[i,:missing_num]," & ")
        s *= string(df[i,:missing_cat]," & ")
        s *= string(df[i,:Num_Patterns]," & ")
        s *= string(df[i,:p_num], " & ")
        try 
            target_list = names(CSV.read("../datasets/"*dname*"/Y.csv", missingstrings=["", "NaN"], DataFrame))

            Y = []
            if length(target_list) <= 2
                target_name = setdiff(target_list, [:Id])[1]
                Y = CSV.read("../datasets/"*dname*"/Y.csv", missingstrings=["", "NaN"], DataFrame)[:,target_name]
            else
                setdiff!(target_list, [:Id, :target])
                sort!(target_list)
                Y = CSV.read("../datasets/"*dname*"/Y.csv", missingstrings=["", "NaN"], DataFrame)[:,target_list[1]]
            end
            if eltype(Y) ∉ [Float64, Int64, Union{Float64,Missing}, Union{Int64,Missing}]
                using StatsBase
                cm = countmap(Y)
                level = collect(keys(cm))[argmax(collect(values(cm)))]
                Y = 1.0 .* (Y .== level)
            end
            ind_availtarget = .!ismissing.(Y)
            Y = 1.0 .* Y[ind_availtarget] #Remove missing entries before converting to Float64 !

            s *= string(length(unique(Y)) == 2 ? "bin." : "cont.")

        catch 
            s *= string("\\texttt{NA}")
        end 

        s *= " \\\\ \n"
    end
end
print(s)

auto-mpg & 398 & 13 & 1 & 0 & 2 & 13 & cont. \\ 
breast-cancer-wisconsin-original & 699 & 9 & 1 & 0 & 2 & 9 & bin. \\ 
breast-cancer-wisconsin-prognostic & 198 & 32 & 1 & 0 & 2 & 32 & bin. \\ 
dermatology & 366 & 130 & 1 & 0 & 2 & 130 & cont. \\ 
ggplot2-movies & 58788 & 34 & 1 & 0 & 2 & 34 & \texttt{NA} \\ 
indian-liver-patient & 583 & 11 & 1 & 0 & 2 & 11 & bin. \\ 
rpart-car.test.frame & 60 & 81 & 1 & 0 & 2 & 81 & bin. \\ 
Ecdat-MCAS & 180 & 13 & 2 & 0 & 3 & 13 & cont. \\ 
MASS-Cars93 & 93 & 64 & 2 & 0 & 3 & 64 & cont. \\ 
car-Davis & 200 & 6 & 2 & 0 & 4 & 6 & \texttt{NA} \\ 
car-Freedman & 110 & 4 & 2 & 0 & 2 & 4 & \texttt{NA} \\ 
car-Hartnagel & 37 & 8 & 2 & 0 & 2 & 8 & \texttt{NA} \\ 
datasets-airquality & 153 & 4 & 2 & 0 & 4 & 4 & \texttt{NA} \\ 
mlmRev-Gcsemv & 1905 & 77 & 2 & 0 & 3 & 77 & \texttt{NA} \\ 
MASS-Pima.tr2 & 300 & 7 & 3 & 0 & 6 & 7 & bin. \\ 
Ecdat-RetSchool & 3078 & 37 & 4 & 0 & 8 & 37 & cont. \\ 
arrhythmia & 452 & 391 & 5 & 0 & 7 & 391 & cont. \\ 
boot-neuro & 46

┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead

In [9]:
s = ""
for i in 1:nrow(df)
    dname = df[i,:Name]
    if !( df[i,:missing_cat] == 0 || df[i,:missing_num] == 0)
        s *= df[i,:Name]*" & "
        s *= string(df[i,:n]," & ")
        s *= string(df[i,:p_all]," & ")
        s *= string(df[i,:missing_num]," & ")
        s *= string(df[i,:missing_cat]," & ")
        s *= string(df[i,:Num_Patterns]," & ")
        s *= string(df[i,:p_num], " & ")
        try 
            target_list = names(CSV.read("../datasets/"*dname*"/Y.csv", missingstrings=["", "NaN"], DataFrame))

            Y = []
            if length(target_list) <= 2
                target_name = setdiff(target_list, [:Id])[1]
                Y = CSV.read("../datasets/"*dname*"/Y.csv", missingstrings=["", "NaN"], DataFrame)[:,target_name]
            else
                setdiff!(target_list, [:Id, :target])
                sort!(target_list)
                Y = CSV.read("../datasets/"*dname*"/Y.csv", missingstrings=["", "NaN"], DataFrame)[:,target_list[1]]
            end
            if eltype(Y) ∉ [Float64, Int64, Union{Float64,Missing}, Union{Int64,Missing}]
                using StatsBase
                cm = countmap(Y)
                level = collect(keys(cm))[argmax(collect(values(cm)))]
                Y = 1.0 .* (Y .== level)
            end
            ind_availtarget = .!ismissing.(Y)
            Y = 1.0 .* Y[ind_availtarget] #Remove missing entries before converting to Float64 !

            s *= string(length(unique(Y)) == 2 ? "bin." : "cont.")

        catch 
            s *= string("\\texttt{NA}")
        end 

        s *= " \\\\ \n"
    end
end
print(s)

pscl-politicalInformation & 1800 & 1440 & 1 & 1431 & 3 & 1441 & bin. \\ 
car-SLID & 7425 & 8 & 2 & 3 & 8 & 9 & \texttt{NA} \\ 
rpart-stagec & 146 & 15 & 2 & 3 & 4 & 16 & \texttt{NA} \\ 
Ecdat-Schooling & 3010 & 51 & 2 & 8 & 9 & 53 & cont. \\ 
mammographic-mass & 961 & 15 & 2 & 13 & 9 & 18 & bin. \\ 
cluster-plantTraits & 136 & 68 & 2 & 37 & 16 & 85 & \texttt{NA} \\ 
mlmRev-star & 24613 & 122 & 2 & 72 & 19 & 128 & cont. \\ 
car-Chile & 2532 & 14 & 3 & 3 & 7 & 15 & bin. \\ 
heart-disease-hungarian & 294 & 25 & 3 & 17 & 16 & 31 & bin. \\ 
heart-disease-switzerland & 123 & 26 & 3 & 18 & 12 & 32 & bin. \\ 
ggplot2-msleep & 83 & 35 & 3 & 29 & 15 & 37 & \texttt{NA} \\ 
survival-cancer & 228 & 13 & 4 & 4 & 8 & 14 & cont. \\ 
heart-disease-va & 200 & 25 & 4 & 13 & 18 & 30 & bin. \\ 
MASS-survey & 237 & 24 & 4 & 19 & 8 & 29 & \texttt{NA} \\ 
hepatitis & 155 & 32 & 5 & 20 & 21 & 42 & bin. \\ 
automobile & 205 & 69 & 6 & 2 & 7 & 70 & cont. \\ 
echocardiogram & 132 & 8 & 6 & 2 & 13 & 9 & bin. \\ 
t

┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV /Users/jpauph/.julia/packages/CSV/jFiCn/src/context.jl:266
┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead